# Google Gemini API: Code Execution Tool Demo

This notebook demonstrates the use of Google's Gemini API code execution feature, which enables the model to generate and run Python code to solve problems iteratively.

## Overview

The code execution tool allows Gemini to:
- Generate Python code to solve problems
- Execute the code in a sandboxed environment
- Learn from execution results iteratively
- Return code, execution results, and summaries

**Key Features:**
- 🔢 Mathematical computations and problem solving
- 📊 Data analysis and visualization (with Matplotlib, Pandas, etc.)
- 🧮 Symbolic math with SymPy
- 📈 Scientific computing with NumPy, SciPy, scikit-learn
- 💾 File processing (CSV, text files, images)

## Prerequisites

```bash
pip install google-genai
```

## Setup and Configuration

In [ ]:
# Install the required package
!pip install -q google-genai

In [ ]:
import os
from google import genai
from google.genai import types

# Set your API key (get it from https://aistudio.google.com/app/apikey)
# Option 1: Set as environment variable
# os.environ['GOOGLE_API_KEY'] = 'your-api-key-here'

# Option 2: Pass directly to client
# client = genai.Client(api_key='your-api-key-here')

# Option 3: Load from environment
client = genai.Client()

print("✓ Gemini API client initialized")

## Example 1: Basic Code Execution - Prime Numbers

Let's start with a classic example: calculating the sum of the first 50 prime numbers.

In [ ]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What is the sum of the first 50 prime numbers? "
             "Generate and run code for the calculation, and make sure you get all 50.",
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

# Display all parts of the response
for i, part in enumerate(response.candidates[0].content.parts):
    if part.text is not None:
        print(f"\n{'='*60}")
        print(f"TEXT RESPONSE (Part {i+1})")
        print(f"{'='*60}")
        print(part.text)
    
    if part.executable_code is not None:
        print(f"\n{'='*60}")
        print(f"GENERATED CODE (Part {i+1})")
        print(f"{'='*60}")
        print(part.executable_code.code)
    
    if part.code_execution_result is not None:
        print(f"\n{'='*60}")
        print(f"EXECUTION RESULT (Part {i+1})")
        print(f"{'='*60}")
        print(part.code_execution_result.output)

## Example 2: Mathematical Problem Solving - Fibonacci Sequence

Let's solve a more complex mathematical problem.

In [ ]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Find the sum of all even-valued Fibonacci numbers that do not exceed 4 million. "
             "Show your work with code.",
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

print(response.text)
print("\n" + "="*60)
print("USAGE METADATA")
print("="*60)
print(f"Prompt tokens: {response.usage_metadata.prompt_token_count}")
print(f"Candidates tokens: {response.usage_metadata.candidates_token_count}")
print(f"Total tokens: {response.usage_metadata.total_token_count}")

## Example 3: Statistical Analysis with NumPy and Pandas

Demonstrate data analysis capabilities using the included libraries.

In [ ]:
prompt = """
Generate a dataset of 1000 random numbers from a normal distribution with mean 100 and 
standard deviation 15. Then:
1. Calculate the mean, median, mode, and standard deviation
2. Find the 25th, 50th, and 75th percentiles
3. Count how many values fall within 1, 2, and 3 standard deviations from the mean

Use NumPy and Pandas for the calculations.
"""

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

print(response.text)

## Example 4: Symbolic Mathematics with SymPy

Solve calculus and algebra problems symbolically.

In [ ]:
prompt = """
Using SymPy, solve the following:
1. Find the derivative of f(x) = x^3 * sin(x) * e^x
2. Integrate g(x) = x^2 * cos(x) from 0 to π
3. Solve the equation: x^2 + 5x + 6 = 0
4. Find the limit of (sin(x)/x) as x approaches 0

Show each step clearly.
"""

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

print(response.text)

## Example 5: Data Visualization

Note: With Gemini 2.0 Flash and later models, code execution supports Matplotlib graph output as inline images.

In [ ]:
prompt = """
Create a visualization showing:
1. A scatter plot of 100 random points from two different normal distributions
2. Add a trend line to show the correlation
3. Include appropriate labels, title, and legend

Use matplotlib and numpy. Make it visually appealing with colors.
"""

response = client.models.generate_content(
    model="gemini-2.0-flash-exp",  # Use 2.0 Flash for graph output
    contents=prompt,
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

# Display the response
for part in response.candidates[0].content.parts:
    if part.text:
        print(part.text)
    if part.executable_code:
        print("\n--- Generated Code ---")
        print(part.executable_code.code)
    if part.code_execution_result:
        print("\n--- Execution Result ---")
        print(part.code_execution_result.output)

## Example 6: Chat with Code Execution

Use code execution in a multi-turn conversation.

In [ ]:
# Create a chat session with code execution enabled
chat = client.chats.create(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

# First message
response = chat.send_message("Hi! I have some math and data analysis questions.")
print("Assistant:", response.text)
print("\n" + "="*60 + "\n")

# Second message - ask for calculation
response = chat.send_message(
    "Calculate the factorial of 20 and then find what percentage of that is 10^18"
)
print("Assistant:", response.text)
print("\n" + "="*60 + "\n")

# Third message - follow-up
response = chat.send_message(
    "Now calculate how many trailing zeros are in factorial of 100"
)
print("Assistant:", response.text)

## Example 7: Machine Learning with Scikit-learn

Demonstrate ML capabilities using the included libraries.

In [ ]:
prompt = """
Create a simple machine learning example:
1. Generate a synthetic dataset for binary classification (200 samples, 2 features)
2. Split it into training (80%) and testing (20%) sets
3. Train a logistic regression model
4. Calculate and display the accuracy, precision, recall, and F1 score
5. Show the confusion matrix

Use scikit-learn for all ML operations.
"""

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

print(response.text)

## Example 8: Complex Problem - Monte Carlo Simulation

Use code execution for a more sophisticated computational task.

In [ ]:
prompt = """
Perform a Monte Carlo simulation to estimate π (pi):
1. Generate 1 million random points in a 1x1 square
2. Count how many fall within a quarter circle (radius 1)
3. Use the ratio to estimate π
4. Calculate the error compared to the actual value of π
5. Show how the estimate improves with more samples (plot convergence)

Use NumPy for efficient computation.
"""

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

print(response.text)

## Example 9: Text Processing and Analysis

In [ ]:
prompt = """
Analyze the following text and provide statistics:

"The quick brown fox jumps over the lazy dog. The dog was not amused. 
The fox, however, was quite pleased with itself. Dogs and foxes have 
an interesting relationship in nature. Some dogs chase foxes, while 
others ignore them completely."

Calculate and show:
1. Total word count
2. Unique word count
3. Most frequent words (top 5)
4. Average word length
5. Sentence count
6. Character count (excluding spaces)
"""

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

print(response.text)

## Example 10: Combinatorics and Optimization

In [ ]:
prompt = """
Solve this optimization problem:

A company produces two products, A and B. 
- Product A gives $50 profit and requires 2 hours of labor
- Product B gives $40 profit and requires 1 hour of labor
- There are 100 hours of labor available
- They can produce at most 40 units of product A
- They can produce at most 60 units of product B

Find the optimal production quantities to maximize profit.
Use linear programming with scipy.optimize.linprog
"""

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

print(response.text)

## Helper Function: Clean Display of Responses

Create a utility function to nicely display code execution results.

In [ ]:
def display_code_execution_response(response):
    """
    Display a Gemini code execution response in a formatted way.
    """
    from IPython.display import display, Markdown, Code
    
    for i, part in enumerate(response.candidates[0].content.parts):
        if part.text is not None:
            display(Markdown(f"### 📝 Response (Part {i+1})"))
            display(Markdown(part.text))
            print()
        
        if part.executable_code is not None:
            display(Markdown(f"### 💻 Generated Code (Part {i+1})"))
            display(Code(part.executable_code.code, language='python'))
            print()
        
        if part.code_execution_result is not None:
            display(Markdown(f"### ✅ Execution Result (Part {i+1})"))
            print(part.code_execution_result.output)
            print()

# Example usage
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Calculate the first 10 Fibonacci numbers and their sum.",
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

display_code_execution_response(response)

## Understanding Token Usage and Billing

Let's examine how tokens are counted with code execution.

In [ ]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Calculate 15 factorial and show the result.",
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

print("Response:", response.text)
print("\n" + "="*60)
print("TOKEN USAGE BREAKDOWN")
print("="*60)
print(f"📥 Prompt tokens: {response.usage_metadata.prompt_token_count}")
print(f"📤 Candidates tokens: {response.usage_metadata.candidates_token_count}")
print(f"📊 Total tokens: {response.usage_metadata.total_token_count}")

print("\n💡 Billing Info:")
print("   - You're billed for input tokens (your prompt)")
print("   - Generated code counts as output tokens")
print("   - Code execution results count as output tokens")
print("   - Final summary counts as output tokens")

## Supported Libraries

The code execution environment includes many useful libraries:

In [ ]:
supported_libraries = [
    "numpy", "pandas", "matplotlib", "seaborn", "scipy", "scikit-learn",
    "sympy", "tensorflow", "pillow", "opencv-python", "imageio",
    "geopandas", "openpyxl", "python-docx", "python-pptx", "PyPDF2",
    "reportlab", "fpdf", "pylatex", "lxml", "jsonschema", "chess",
    "tabulate", "joblib", "jinja2", "mpmath", "attrs", "toolz"
]

print("📚 Available Libraries in Code Execution Environment:\n")
for i, lib in enumerate(supported_libraries, 1):
    print(f"{i:2d}. {lib}")

print("\n⚠️  Note: You cannot install custom libraries")

## Key Limitations and Considerations

1. **Language**: Only Python code can be executed
2. **Runtime**: Maximum 30 seconds execution time
3. **Retries**: Up to 5 automatic retries if code generates an error
4. **Libraries**: Cannot install custom libraries (fixed set available)
5. **Output**: Code can't return media files, only text/data
6. **Model variations**: Different models may have varying success rates
7. **File I/O**: Gemini 2.0+ supports CSV/text input and graph output (up to 1M tokens ~2MB)

## Best Practices

1. **Be specific**: Clearly describe what you want the code to do
2. **Request verification**: Ask the model to verify results or show intermediate steps
3. **Use appropriate models**: Gemini 2.5 Flash for general use, 2.0 Flash Exp for graphs
4. **Monitor tokens**: Code execution can increase token usage significantly
5. **Leverage libraries**: Use the included libraries for specialized tasks
6. **Iterative refinement**: Use chat mode for complex multi-step problems

## Additional Resources

- 📖 [Official Documentation](https://ai.google.dev/gemini-api/docs/code-execution)
- 🔬 [Code Execution Colab](https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Code_Execution.ipynb)
- 🔑 [Get API Key](https://aistudio.google.com/app/apikey)
- 📚 [Gemini API Docs](https://ai.google.dev/gemini-api/docs)
- 💰 [Pricing Information](https://ai.google.dev/pricing)

## Conclusion

The Gemini API code execution tool is a powerful feature that enables:
- Complex mathematical computations
- Data analysis and visualization
- Scientific computing
- Machine learning tasks
- Text and file processing

All without setting up a separate code execution environment! 🚀